In [ ]:
get_ipython().run_cell_magic('capture', '', "%logstop\n%logstart -t -r -q ipython_command_log.py global\n\n#- IRONHACKS RESEARCH TRACKING CODE\n#----------------------------------\n# The following code is used to help our research team understand how you \n# our notebook environment. We do not collect any personal information with\n# the following code, it is used to measure when and how often you work on\n# your submission files.\n\nimport os\nfrom datetime import datetime\nimport IPython.core.history as history\n\nha = history.HistoryAccessor()\nha_tail = ha.get_tail(1)\nha_cmd = next(ha_tail)\nsession_id = str(ha_cmd[0])\ncommand_id = str(ha_cmd[1])\ntimestamp = datetime.utcnow().isoformat()\nhistory_line = ','.join([session_id, command_id, timestamp]) + '\\n'\nlogfile = open(os.environ['HOME']+'/ipython_session_log.csv', 'a')\nlogfile.write(history_line)\nlogfile.close()\n")

In [ ]:
get_ipython().run_cell_magic('capture', '', 'import pandas as pd\nimport numpy as np\nimport os\nfrom google.cloud import bigquery\nfrom google.oauth2 import service_account\nfrom google.cloud.bigquery import magics\nimport matplotlib.pyplot as plt\nimport seaborn as sns\nfrom sklearn.svm import SVR\nfrom sklearn.model_selection import GridSearchCV\nfrom sklearn.metrics import mean_absolute_error, mean_squared_error\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.ensemble import RandomForestRegressor\nfrom sklearn.linear_model import LassoCV\nfrom sklearn.model_selection import RepeatedKFold\nfrom sklearn.linear_model import BayesianRidge\nfrom sklearn.linear_model import ElasticNetCV\nfrom keras.models import Sequential\nfrom keras.layers import Bidirectional, LSTM, Dropout, Dense\nfrom keras.models import load_model\nimport joblib\nfrom joblib import Parallel, delayed\n')

In [ ]:
def evaluate_regressor(prediction_dataframe):
    # Takes in a prediction dataframe of 2 columns, Actual values and Predicted values generated by a regressor
    # Outputs MSE, MAR, RMSE and MAPE metrics. Must have columns named Actual and Predicted.
    print('MSE:', mean_squared_error(prediction_dataframe['Actual'], prediction_dataframe['Predicted']))
    print('MAE:', mean_absolute_error(prediction_dataframe['Actual'], prediction_dataframe['Predicted']))
    print('RMSE:', np.sqrt(mean_squared_error(prediction_dataframe['Actual'], prediction_dataframe['Predicted'])))
    print('MAPE:', np.mean(np.abs((prediction_dataframe['Actual'] - prediction_dataframe['Predicted']) / prediction_dataframe['Actual'])) * 100)

In [ ]:
def get_predictions(regressor, model_type, name, week):
    # generates predictions for any model and writes out a dataframe in csv containing them
    # takes a regressor and learning method type as input: DL and ML
    # DL/ML variable basically changes the shape for an input from a 2D array to 3D arry, as required tensor shape
    result_list = []
    uu_id_transform = LE.fit_transform(prediction_list['uu_id'])
    if model_type == 'DL':
        predict_arr = np.array(SC_other.transform([[-0.04, -0.140, 0.328, -0.671, -0.420, -0.432, -0.0013, -0.0023, -0.347, -0.0004, 3.211, -0.532, -0.329]]))
        for val in uu_id_transform:
            to_predict = np.insert(predict_arr, 0, val, axis=1)
            to_predict = np.insert(to_predict, 1, week, axis=1)
            to_predict = np.reshape(to_predict, (to_predict.shape[0], to_predict.shape[1],1))
            r = regressor.predict(to_predict)
            result_list.append(r)
        result_list = np.array(result_list)
        result_list = np.reshape(result_list, (525,))
    elif model_type == 'ML':
        predict_arr = np.array(RB_other.transform([[-0.04, -0.140, 0.328, -0.671, -0.420, -0.432, -0.0013, -0.0023, -0.347, -0.0004, 3.211, -0.532, -0.329]]))
        for val in uu_id_transform:
            to_predict = np.insert(predict_arr, 0, val, axis=1)
            to_predict = np.insert(to_predict, 1, week, axis=1)
            r = regressor.predict(to_predict)
            result_list.append(r)
    result_df = pd.DataFrame(result_list, columns = ['Predictions'])
    prediction_sub = prediction_list.copy()
    prediction_sub['total_claims'] = result_df.values
    prediction_sub = prediction_sub[['uu_id','total_claims','week_number']]
    os.makedirs('lost+found/submission_files', exist_ok=True)
    prediction_sub.to_csv('lost+found/submission_files/'+name+'.csv', index=False)
    return prediction_sub

In [ ]:
def get_pred_frame(test_frame, prediction_array):
    prediction_frame = pd.DataFrame({'Actual': test_frame, 'Predicted': prediction_array.flatten()})
    return prediction_frame

In [ ]:
# updated_ingest = pd.concat([merged_ingest, combined_ingest])
ingest = pd.read_csv('lost+found/submission_files/complete_ingest.csv')

In [ ]:
ingest.shape

In [ ]:
ingest.isnull().sum()

In [ ]:
ingest.dropna(axis=0, inplace=True)
print(ingest.shape)

In [ ]:
# Based on RFR results, there appears to be an issue with outliers - TODO: Find and remove them
ingest.columns